In [1]:
import imageio
import numpy as np
import cv2
import json
import torch
from scipy.spatial.transform import Rotation as scipy_rot
import sys
from torchvision.models import vgg16
from PIL import Image
from matplotlib import pyplot as plt
import itertools
import k3d
from pytorch3d.io import load_objs_as_meshes, save_obj, load_obj, save_ply
import pandas as pd
from leveraging_geometry_for_shape_estimation.eval.scannet.CSVHelper import read as read_csv

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
def load_mesh(shapenet_path,category,model_id,r,t,s):

    vertices,faces,properties = load_obj('{}/{}/{}/model_normalized.obj'.format(shapenet_path,category.replace('bookcase','bookshelf'),model_id),create_texture_atlas=False, load_textures=False)

    vertices = vertices.cpu().numpy()
    vertices_scaled = np.array(s) * vertices
    vertices =  np.matmul(np.array(r),vertices_scaled.T).T + np.array(t)
    return vertices,faces[0].numpy()



In [3]:
def writePlyFile(file_name, vertices, colors):
    ply_header = '''ply
                format ascii 1.0
                element vertex %(vert_num)d
                property float x
                property float y
                property float z
                property uchar red
                property uchar green
                property uchar blue
                end_header
               '''
    print(vertices.shape)
    print(colors.shape)
    vertices = vertices.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    vertices = np.hstack([vertices, colors])
    with open(file_name, 'w') as f:
      f.write(ply_header % dict(vert_num=len(vertices)))
      np.savetxt(f, vertices, '%f %f %f %d %d %d')

In [4]:
with open('/scratch2/fml35/results/ROCA/debug_evaluation/gt_scene0011_00_bin.json','r') as f:
    gt = json.load(f)

with open('/scratch2/fml35/results/ROCA/debug_evaluation/single_pred_scene0011_00_bin.json','r') as f:
    pred = json.load(f)
shapenet_path = '/scratch2/fml35/datasets/shapenet_v2/ShapeNetRenamed/model'
# plot = k3d.plot()

# vert_pred,face_pred = load_mesh(shapenet_path,pred["category"],pred["id_cad"],pred["r"],pred["t"],pred["s"])
# plot += k3d.mesh(vert_pred,face_pred,color=(255 + 128 * 256))

# all_verts = [vert_pred]

# for i in range(len(gt["category"])):
#     vert_gt,face_gt = load_mesh(shapenet_path,gt["category"][i],gt["id_cad"][i],gt["r"][i],gt["t"][i],gt["s"][i])
#     plot += k3d.mesh(vert_gt,face_gt,color=(64 + 128 * 256))
#     all_verts.append(vert_gt)

# stacked = np.concatenate(all_verts)
# stacked = torch.Tensor(stacked)

# save_ply('/scratch2/fml35/results/ROCA/debug_evaluation/test_vis.ply',stacked)

# plot.display()

In [22]:
def plot_scene(scene,plot_scene=False):
    with open('/scratch2/fml35/results/ROCA/results_per_scene_own_2_reformated/{}.json'.format(scene),'r') as f:
        annotations = json.load(f)

    with open('/scratch2/fml35/results/ROCA/results_per_scene_reformated/{}.json'.format(scene),'r') as f:
        annotations_filtered = json.load(f)

    with open('/scratch2/fml35/results/ROCA/gt_per_scene/{}.json'.format(scene),'r') as f:
        annotations_gt = json.load(f)
    print('N',len(annotations),"N filtered ",len(annotations_filtered))
    plot = k3d.plot()
    distances = []

    all_verts_colors = {'verts':[],'colors':[]}


    for i in range(len(annotations)):
        print(annotations[i])
        if not annotations[i]["id_cad"] == 'b1da0d9aab6d2308608505d960f2a393' or not annotations[i]["score"] == 0.9721203446388245:
            continue
        vert,face = load_mesh(shapenet_path,annotations[i]["category"],annotations[i]["id_cad"],annotations[i]["r"],annotations[i]["t"],annotations[i]["s"])

        plot += k3d.mesh(vert,face,color=(128 * 256*256))
        all_verts_colors["verts"] = all_verts_colors["verts"] + list(vert)
        all_verts_colors["colors"] = all_verts_colors["colors"] + [[0,255,0] for v in list(vert)]
    
    for i in range(len(annotations_filtered)):
        if not annotations[i]["id_cad"] == 'b1da0d9aab6d2308608505d960f2a393' or not annotations[i]["score"] == 0.9721203446388245:
            continue

        vert,face = load_mesh(shapenet_path,annotations_filtered[i]["category"],annotations_filtered[i]["id_cad"],annotations_filtered[i]["r"],annotations_filtered[i]["t"],annotations_filtered[i]["s"])
        plot += k3d.mesh(vert,face,color=(255 + 128 * 256))

        dist = np.array(annotations_filtered[i]["t"])-np.array(annotations[i]["t"])
        distances.append(dist)
        all_verts_colors["verts"] = all_verts_colors["verts"] + list(vert)
        
        all_verts_colors["colors"] = all_verts_colors["colors"] + [[255,0,0] for v in list(vert)]
        # print('dist',np.array(annotations_filtered[i]["t"])-np.array(annotations[i]["t"]))
    for i in range(len(annotations_gt)):
        if not annotations[i]["id_cad"] == 'b1da0d9aab6d2308608505d960f2a393':
            continue
        vert,face = load_mesh(shapenet_path,annotations_gt[i]["category"],annotations_gt[i]["id_cad"],annotations_gt[i]["r"],annotations_gt[i]["t"],annotations_gt[i]["s"])
        plot += k3d.mesh(vert,face,color=(128 * 256))


    # writePlyFile('/scratch2/fml35/results/ROCA/vis_scenes/{}.ply'.format(scene),np.array(all_verts_colors["verts"]),np.array(all_verts_colors["colors"]))
    distances = np.array(distances)
    diff_distances = distances - distances[0,:]
    magnitude = np.linalg.norm(diff_distances,axis=1)
    # assert np.max(magnitude) < 0.00001, (magnitude)
    print('assertion failed')
    if plot_scene:
        plot.display()

        plot.fetch_screenshot()
        with open('/scratch2/fml35/results/ROCA/vis_scenes/{}.html'.format(scene),'w') as fp:
            fp.write(plot.snapshot)


In [24]:
scene = "scene0621_00"
plot_scene(scene,plot_scene=True)

N 150 N filtered  150
{'category': 'chair', 'id_cad': 'b1da0d9aab6d2308608505d960f2a393', 'r': [[0.8143374001726251, -0.0041868084350061285, 0.5803766615872991], [0.5803749551411186, 0.013484491417706024, -0.8142377293740141], [-0.004417026718767425, 0.999900314594577, 0.013410844441395953]], 't': [3.8290243446265393, 1.1075646004435178, 0.4660326609485912], 's': [1.1075780391693115, 1.1034075021743774, 1.104204535484314]}


KeyError: 'score'

In [26]:
import os
# import random
# random.seed(3)
# Find scenen with not too many objects
scenes = sorted(os.listdir('/scratch2/fml35/results/ROCA/results_per_scene_own_1_reformated'))
# random.shuffle(scenes)
# for i in range(len(scenes)):
#     with open('/scratch2/fml35/results/ROCA/results_per_scene_own_1_reformated/' + scenes[i],'r') as f:
#         annotations = json.load(f)
#     scene = scenes[i].split('.')[0]
#     if len(annotations) < 20:
#         break
scene = "scene0621_00"
# print(scene)
# print('NOT FILTERED')
for scene_file in scenes:
    scene = scene_file.split('.')[0]
    print(scene)
    plot_scene(scene,plot_scene=True)
    #break



scene0011_00
N 83 N filtered  83
(90062, 3)
(90062, 3)
scene0011_01
N 94 N filtered  94
(19208, 3)
(19208, 3)
scene0015_00
N 103 N filtered  103


In [57]:
with open('/scratch2/fml35/results/ROCA/gt_per_scene/{}.json'.format(scene),'r') as f:
    annotations = json.load(f)

plot = k3d.plot()
for i in range(len(annotations)):
    vert,face = load_mesh(shapenet_path,annotations[i]["category"],annotations[i]["id_cad"],annotations[i]["r"],annotations[i]["t"],annotations[i]["s"])
    plot += k3d.mesh(vert,face,color=(64 + 128 * 256))
plot.display()

[-0.38479394 -2.75659412 -0.28431616]

Output()

In [33]:
# with open('/scratch2/fml35/results/ROCA/results_per_scene_filtered_reformated/scene0011_00.json','r') as f:
#     annotations = json.load(f)

# plot = k3d.plot()
# for i in range(len(annotations)):
#     vert,face = load_mesh(shapenet_path,annotations[i]["category"],annotations[i]["id_cad"],annotations[i]["r"],annotations[i]["t"],annotations[i]["s"])
#     plot += k3d.mesh(vert,face,color=(64 + 128 * 256))
# plot.display()
# print('N',len(annotations))

/home/mifs/fml35/environments/shape_env/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/mifs/fml35/environments/shape_env/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

N 52
